In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
import pandas as pd

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
import boto3
AWS_ACCESS_KEY_ID= config['AWS']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY= config['AWS']['AWS_SECRET_ACCESS_KEY']
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=AWS_ACCESS_KEY_ID,
                       aws_secret_access_key=AWS_SECRET_ACCESS_KEY
                     )

In [4]:
sampleDbBucket =  s3.Bucket("udacity-dend")

In [5]:
# for obj in sampleDbBucket.objects.all():
#     print(obj)
for obj in sampleDbBucket.objects.filter(Prefix="song_data/A/A/A/"):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAEA128F935A30D.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAED128E0783FAB.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAEM128F93347B9.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAEW128F42930C0.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAFD128F92F423A.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAGR128F425B14B.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/A/A/A/TRAAAHD128F426

In [6]:
print(os.environ['AWS_SECRET_ACCESS_KEY'])

OBZvoZaYGd0/uJI90TVyJoJMswhRgH5sdd1BZks1


In [7]:
spark = SparkSession.builder\
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config("spark.hadoop.fs.s3a.multiobjectdelete.enable","false") \
        .config("spark.hadoop.fs.s3a.fast.upload","true") \
        .getOrCreate()

# spark.sparkContext.hadoopConfiguration.set("fs.s3a.multiobjectdelete.enable","false")
# spark.sparkContext.hadoopConfiguration.set("fs.s3a.access.key","my access key")
# spark.sparkContext.hadoopConfiguration.set("fs.s3a.secret.key","my secret key")

# spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.multiobjectdelete.enable","false")

In [8]:
# spark.sparkContext.getConf().getAll()

In [9]:
input_data = "s3a://udacity-dend/"

In [10]:
# get filepath to song data file
song_data = os.path.join(input_data, 'song_data/A/A/A/*.json')
print(song_data)

s3a://udacity-dend/song_data/A/A/A/*.json


In [11]:
# read song data file
df_song = spark.read.json(song_data)

In [12]:
# df_song.limit(10).toPandas()

In [13]:
# df_song.count()

In [14]:
# extract columns to create songs table
songs_table = df_song.select('song_id', 'title', 'artist_id', 'year', 'duration').dropDuplicates()
# songs_table.show()

In [15]:
# songs_table.count()

In [16]:
songs_table.printSchema()

root
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- year: long (nullable = true)
 |-- duration: double (nullable = true)



In [17]:
pd.set_option('max_colwidth',200)
# songs_table.toPandas()

In [18]:
import shutil
try:
    shutil.rmtree("artists") 
    shutil.rmtree("songs")
    shutil.rmtree("users")
    shutil.rmtree("time")
    shutil.rmtree("songplays")
except:
    pass

In [19]:
output_data = "s3a://lcw-udacity-data-lake-west/"

In [20]:
songs_data= os.path.join(output_data, "songs")
songs_data

's3a://lcw-udacity-data-lake-west/songs'

In [21]:
%%time
# write songs table to parquet files partitioned by year and artist
songs_table.write.partitionBy("year", "artist_id").parquet(songs_data, mode="overwrite")

CPU times: user 32.4 ms, sys: 15.4 ms, total: 47.8 ms
Wall time: 4min 24s


In [22]:

# extract columns to create artists table
artists_table = df_song.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude').dropDuplicates()
artists_table.printSchema()
# artists_table.toPandas()


root
 |-- artist_id: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_longitude: double (nullable = true)



In [23]:
%%time
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data + "artists")

CPU times: user 30.1 ms, sys: 11.6 ms, total: 41.7 ms
Wall time: 3min 52s


In [24]:
# get filepath to log data file
log_data = "data/log_data/*.json"

In [25]:
# read log data file
df_log = spark.read.json(log_data)
df_log.limit(5).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
3,None,Logged In,Wyatt,M,0,Scott,NaN,free,"Eureka-Arcata-Fortuna, CA",GET,Home,1.540872e+12,563,None,200,1542247071796,Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko,9
4,None,Logged In,Austin,M,0,Rosales,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.541060e+12,521,None,200,1542252577796,Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20100101 Firefox/31.0,12


In [26]:
# filter by actions for song plays
df_log = df_log.filter(df_log.page=='NextSong')
df_log.limit(3).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26
2,Train,Logged In,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Marry Me,200,1542242741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26


In [27]:
# extract columns for users table    
users_table = df_log.selectExpr('userId as user_id', 'firstName as first_name', 'lastName as last_name', 'gender', 'level').dropDuplicates()

In [28]:
users_table.limit(3).toPandas()

,user_id,first_name,last_name,gender,level
0,26,Ryan,Smith,M,free
1,7,Adelyn,Jordan,F,free
2,71,Ayleen,Wise,F,free


In [29]:
%%time
# write users table to parquet files
users_table.write.mode('overwrite').parquet(output_data + "users")

CPU times: user 88.8 ms, sys: 29.1 ms, total: 118 ms
Wall time: 11min 13s


In [30]:
df_log.head()

Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26')

In [31]:

# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: int(x) / 1000.0)
df_log = df_log.withColumn("timestamp", get_timestamp(df_log.ts))


In [32]:
df_log.limit(2).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,1.542241826796E9
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,1.542242481796E9


In [33]:
get_datetime = udf(lambda x: str(datetime.fromtimestamp(int(x) / 1000.0)))
df_log = df_log.withColumn("datetime", get_datetime(df_log.ts))

In [34]:
df_log.head()

Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', timestamp='1.542241826796E9', datetime='2018-11-15 00:30:26.796000')

In [35]:
df_log.limit(2).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,datetime
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,1.542241826796E9,2018-11-15 00:30:26.796000
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,1.542242481796E9,2018-11-15 00:41:21.796000


In [36]:
df_log.createOrReplaceTempView("df_log")
# df_song.createOrReplaceTempView("df_song")

In [37]:
time_table = spark.sql("""
                        SELECT 
                           datetime as start_time,
                           EXTRACT(hour from datetime) AS hour, 
                           EXTRACT(day from datetime) AS day, 
                           EXTRACT(week from datetime) AS week, 
                           EXTRACT(month from datetime) AS month, 
                           EXTRACT(year from datetime) AS year, 
                           EXTRACT(dayofweek from datetime) AS weekday
                        FROM df_log
                        """)

In [38]:
time_table.show()

+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:41:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:45:...|   0| 15|  46|   11|2018|      5|
|2018-11-15 03:44:...|   3| 15|  46|   11|2018|      5|
|2018-11-15 05:48:...|   5| 15|  46|   11|2018|      5|
|2018-11-15 05:53:...|   5| 15|  46|   11|2018|      5|
|2018-11-15 05:55:...|   5| 15|  46|   11|2018|      5|
|2018-11-15 06:01:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:07:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:10:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:13:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:14:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:17:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:18:...|   6| 15|  46|   11|2018|      5|
|2018-11-15 06:21:...|   6| 15|  46|   11|2018| 

In [39]:
%%time
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy("year", "month").mode('overwrite').parquet(output_data + "time")

CPU times: user 0 ns, sys: 3.96 ms, total: 3.96 ms
Wall time: 13.4 s


In [40]:
df_log.limit(2).toPandas()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,timestamp,datetime
0,Harmonia,Logged In,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,Sehr kosmisch,200,1542241826796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,1.542241826796E9,2018-11-15 00:30:26.796000
1,The Prodigy,Logged In,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",PUT,NextSong,1.541017e+12,583,The Big Gundown,200,1542242481796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36""",26,1.542242481796E9,2018-11-15 00:41:21.796000


In [41]:
# read in song data to use for songplays table
song_df = spark.read.json('data/song_data/*/*/*/*.json')

In [42]:
song_df.limit(10).toPandas()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARDR4AC1187FB371A1,NaN,,NaN,Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti,511.16363,1,SOBAYLL12A8C138AF9,Sono andati? Fingevo di dormire,0
1,AREBBGV1187FB523D2,NaN,"Houston, TX",NaN,Mike Jones (Featuring CJ_ Mello & Lil' Bran),173.66159,1,SOOLYAZ12A6701F4A6,Laws Patrolling (Album Version),0
2,ARMAC4T1187FB3FA4C,40.82624,"Morris Plains, NJ",-74.47995,The Dillinger Escape Plan,207.77751,1,SOBBUGU12A8C13E95D,Setting Fire to Sleeping Giants,2004
3,ARPBNLO1187FB3D52F,40.71455,"New York, NY",-74.00712,Tiny Tim,43.36281,1,SOAOIBZ12AB01815BE,I Hold Your Hand In Mine [Live At Royal Albert Hall],2000
4,ARDNS031187B9924F0,32.67828,Georgia,-83.22295,Tim Wilson,186.48771,1,SONYPOM12A8C13B2D7,I Think My Wife Is Running Around On Me (Taco Hell),2005
5,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.16200,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994
6,ARLTWXK1187FB5A3F8,32.74863,"Fort Worth, TX",-97.32925,King Curtis,326.00771,1,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),0
7,ARPFHN61187FB575F6,41.88415,"Chicago, IL",-87.63241,Lupe Fiasco,279.97995,1,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),0
8,ARI2JSK1187FB496EF,51.50632,"London, England",-0.12714,Nick Ingman;Gavyn Wright,111.62077,1,SODUJBS12A8C132150,Wessex Loses a Bride,0
9,AR0RCMP1187FB3F427,30.08615,"Beaumont, TX",-94.10158,Billie Jo Spears,133.32853,1,SOGXHEG12AB018653E,It Makes No Difference Now,1992


In [43]:
song_df.createOrReplaceTempView("song_df")

In [44]:
# extract columns from joined song and log datasets to create songplays table 
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
songplays_table = spark.sql("""
                            SELECT row_number() over (order by "somerow") as songplay_id,
                                dl.datetime as start_time, 
                                dl.userId as user_id, 
                                dl.level as user_level, 
                                song_df.song_id,
                                song_df.artist_id, 
                                dl.sessionId as session_id, 
                                dl.location as user_location, 
                                dl.userAgent as user_agent,
                                EXTRACT(year from dl.datetime) AS year,
                                EXTRACT(month from dl.datetime) AS month
                            FROM df_log dl
                            JOIN song_df
                            ON dl.artist = song_df.artist_name and dl.song = song_df.title and dl.length = song_df.duration
                            WHERE dl.page='NextSong'
                            """)


In [45]:
songplays_table.show()

+-----------+--------------------+-------+----------+------------------+------------------+----------+--------------------+--------------------+----+-----+
|songplay_id|          start_time|user_id|user_level|           song_id|         artist_id|session_id|       user_location|          user_agent|year|month|
+-----------+--------------------+-------+----------+------------------+------------------+----------+--------------------+--------------------+----+-----+
|          1|2018-11-21 21:56:...|     15|      paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|2018|   11|
+-----------+--------------------+-------+----------+------------------+------------------+----------+--------------------+--------------------+----+-----+



In [46]:
%%time
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy("year", "month").mode('overwrite').parquet(output_data + "songplays")

CPU times: user 3.47 ms, sys: 596 µs, total: 4.06 ms
Wall time: 12.5 s
